In [1]:
!pip install pandas numpy yfinance statsmodels ta schedule python-dotenv requests openai


In [2]:
import os

PROJECT_PATH = "/content/drive/MyDrive/btc_trading_bot"
os.chdir(PROJECT_PATH)

os.listdir()


['data',
 'features',
 'ml',
 'strategies',
 'execution',
 'llm',
 'reporting',
 'config',
 '.ipynb_checkpoints']

In [3]:
import json, os

#Config JSON file
config = {
    "budget_usd": 10000,
    "dca": {
        "amount": 500,
        "price_drop_pct": 0.03
    },
    "atr": {
        "period": 14,
        "multiplier": 1.5
    },
    "ml": {
        "forecast_horizon": 5
    },
    "strategy_mode": "hybrid"
}

os.makedirs("config", exist_ok=True)
with open("config/strategy_config.json", "w") as f:
    json.dump(config, f, indent=2)

In [4]:
import sys
import os

PROJECT_PATH = "/content/drive/MyDrive/btc_trading_bot"
os.chdir(PROJECT_PATH)
sys.path.append(PROJECT_PATH)

In [5]:
import json
from data.market_data import fetch_btc_data
from features.technicals import add_indicators
from ml.arima_model import arima_forecast
from ml.exp_smoothing import exp_smooth_forecast
from ml.forecast_ensemble import ensemble_forecast
from execution.portfolio import Portfolio
from strategies.hybrid_strategy import HybridStrategy

import json
import os
import numpy as np
import pandas as pd


#Imports
from data.market_data import fetch_btc_data
from features.technicals import add_indicators

from ml.arima_model import arima_forecast
from ml.exp_smoothing import exp_smooth_forecast
from ml.forecast_ensemble import ensemble_forecast

from execution.portfolio import Portfolio
from strategies.hybrid_strategy import HybridStrategy

#Load Config file
with open("config/strategy_config.json") as f:
    config = json.load(f)

print("Loaded config:")
print(json.dumps(config, indent=2))

#Fetch and prep data
print("\nFetching BTC market data...")
df = fetch_btc_data(period="60d", interval="30m")

#Flatten yfinance MultiIndex
df.columns = df.columns.get_level_values(0)

#Reload
import importlib
import features.technicals as tech

# Reload the module to get the latest changes
importlib.reload(tech)

# Then call the function
df = tech.add_indicators(df)

# Safety check
if len(df) < 50:
    raise ValueError("Not enough historical data for ML models")

#ML Forecasting
close_series = df["Close"]

print("\nRunning ARIMA forecast...")
arima_f = arima_forecast(close_series, steps=5)

print("Running Exponential Smoothing forecast...")
exp_f = exp_smooth_forecast(close_series, steps=5)

forecast_series = ensemble_forecast(arima_f, exp_f)
forecast_price = float(forecast_series[0])

print(f"ML Forecast (next step): {forecast_price:.2f}")

#Portfolio and Strategy Initiation
portfolio = Portfolio(config["budget_usd"])
strategy = HybridStrategy(config)

#Latest market context
latest = df.iloc[-1]
previous = df.iloc[-2]

price = float(latest["Close"])
atr = float(latest["ATR"])
rsi = float(latest["RSI"])

volume_change_pct = (
    (latest["Volume"] - previous["Volume"]) / previous["Volume"] * 100
)

print("\nCurrent Market Snapshot:")
print(f"Price: {price:.2f}")
print(f"ATR(14): {atr:.2f}")
print(f"RSI(14): {rsi:.2f}")
print(f"Volume Change %: {volume_change_pct:.2f}")

#Execute hybrid strategy
print("\nRunning Hybrid Strategy...\n")

actions = strategy.run(
    price=price,
    atr=atr,
    forecast=forecast_price,
    rsi=rsi,
    volume_change=volume_change_pct,
    portfolio=portfolio
)

#Output actions
if not actions:
    print("No trades executed this cycle.")
else:
    print("Actions executed:")
    for action in actions:
        print("-", action)

#Print portfolio status
print("\nPortfolio Status:")
print(f"USD Balance: ${portfolio.usd:,.2f}")
print(f"BTC Holdings: {portfolio.btc:.6f} BTC")

portfolio_value = portfolio.usd + portfolio.btc * price
print(f"Total Portfolio Value (USD): ${portfolio_value:,.2f}")



Loaded config:
{
  "budget_usd": 10000,
  "dca": {
    "amount": 500,
    "price_drop_pct": 0.03
  },
  "atr": {
    "period": 14,
    "multiplier": 1.5
  },
  "ml": {
    "forecast_horizon": 5
  },
  "strategy_mode": "hybrid"
}

Fetching BTC market data...


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Running ARIMA forecast...
Running Exponential Smoothing forecast...
ML Forecast (next step): 94876.33

Current Market Snapshot:
Price: 94877.35
ATR(14): 289.18
RSI(14): 51.13
Volume Change %: -2.73

Running Hybrid Strategy...

No trades executed this cycle.

Portfolio Status:
USD Balance: $10,000.00
BTC Holdings: 0.000000 BTC
Total Portfolio Value (USD): $10,000.00


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/py

**Please note:** Multiple test runs have NOT taken place yet, this submission is just to confirm that the logic works as intended and I have an end-to-end system. A few things like reporting and weekly updates are not yet implemented as I was focussing on the primary engine.

In [15]:
#Test run for one trade cycle
from data.market_data import fetch_btc_data
from features.technicals import add_indicators
from execution.portfolio import Portfolio
from strategies.hybrid_strategy import HybridStrategy
from ml.arima_model import arima_forecast
from ml.exp_smoothing import exp_smooth_forecast
from ml.forecast_ensemble import ensemble_forecast

#Load config
config = {
    "budget_usd": 10000,
    "dca": {"amount": 500, "price_drop_pct": 0.03},
    "atr": {"period": 14, "multiplier": 1.5},
    "ml": {"forecast_horizon": 5},
    "strategy_mode": "hybrid"
}

#Initiate portfolio & strategy
portfolio = Portfolio(config["budget_usd"])
strategy = HybridStrategy(config)

#Fetch data
df = fetch_btc_data(period="7d", interval="30m")

# Flatten yfinance MultiIndex to avoid df copy error
df.columns = df.columns.get_level_values(0)

import importlib
import features.technicals as tech

#Reload module to get latest changes
importlib.reload(tech)

df = tech.add_indicators(df)

#Take latest row
latest = df.iloc[-1]
price = latest["Close"]
atr = latest["ATR"]
rsi = latest["RSI"]
volume_change = (latest["Volume"] - df["Volume"].iloc[-2]) / df["Volume"].iloc[-2]

#Forecast with arima and exp_smoothing
arima_f = arima_forecast(df["Close"], steps=1)[0]
exp_f = exp_smooth_forecast(df["Close"], steps=1)[0]
forecast = ensemble_forecast(arima_f, exp_f)

#Hybrid strategy
actions = strategy.run(
    price=price,
    atr=atr,
    forecast=forecast,
    rsi=rsi,
    volume_change=volume_change,
    portfolio=portfolio
)

#Results
print("\nTEST CYCLE RESULTS")
print(f"Market price: {price}")
print(f"ATR: {atr:.2f}, RSI: {rsi:.2f}, Forecast: {forecast:.2f}")
print("Actions taken:")
for act in actions:
    print(" -", act)
print(f"\nPortfolio USD: {portfolio.usd}, BTC: {portfolio.btc}")


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'



TEST CYCLE RESULTS
Market price: 95027.4375
ATR: 282.37, RSI: 53.95, Forecast: 95038.82
Actions taken:

Portfolio USD: 10000, BTC: 0


/tmp/ipython-input-1209919605.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arima_f = arima_forecast(df["Close"], steps=1)[0]
/tmp/ipython-input-1209919605.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exp_f = exp_smooth_forecast(df["Close"], steps=1)[0]
